# Experimental Analysis of Static Community Detection Using M2V

Running both the Louvain and Leiden algorithms multiple times and recording various statistics for each run can provide valuable insights for our analysis.

## Importing Edge List w/ Weights to NetworkX

NetworkX's read_weighted_edgelist function expects a simple text file with lines of the form <node1> <node2> <weight>, without headers. Since our data is in CSV format, you'll need to use Pandas (or another method) to load the CSV and adjust it to become readable.

In [1]:
import pandas as pd

edge_list_df = pd.read_csv('New Input/M2V_edge_list_with_similarity.csv')

print(edge_list_df)

       source  target    weight
0          p0      a1  0.752102
1          p0      a2  0.803198
2          p0      a3  0.900095
3          p0      a4  0.682413
4          p0      a5  0.833705
...       ...     ...       ...
51372  p10169   p8094  0.209173
51373  p10169   p7974  0.235403
51374  p10169   p5852  0.265217
51375  p10169  p10113  0.639329
51376  p10169  p10031  0.371488

[51377 rows x 3 columns]


In [2]:
negative_weights = edge_list_df[edge_list_df['weight'] < 0]
print(f"Number of edges with negative weights: {len(negative_weights)}")

Number of edges with negative weights: 1981


Since Louvain is not made to consider negative edge weights, we will rescale the weights such that instead of [-1, 1] being the range, it is now [0, 1], where 0 now represents perfect dissimilarity, 0.5 represents orthogonality, and 1 represents perfect similarity.

In [3]:
edge_list_df['weight'] = (edge_list_df['weight'] + 1) / 2

print(edge_list_df)

       source  target    weight
0          p0      a1  0.876051
1          p0      a2  0.901599
2          p0      a3  0.950047
3          p0      a4  0.841207
4          p0      a5  0.916853
...       ...     ...       ...
51372  p10169   p8094  0.604586
51373  p10169   p7974  0.617701
51374  p10169   p5852  0.632609
51375  p10169  p10113  0.819664
51376  p10169  p10031  0.685744

[51377 rows x 3 columns]


Before we continue with the creation of a graph, NetworkX specifies that an undirected, weighted graph must not have self-loop, parallel edges (A->B, B->A), or duplicate edges.

In [4]:
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)
print(f"Number of duplicate edges: {duplicate_edges.sum()}")

self_loops = edge_list_df[edge_list_df['source'] == edge_list_df['target']]
print(f"Number of self-loops: {len(self_loops)}")

print(edge_list_df.isnull().sum())

Number of duplicate edges: 0
Number of self-loops: 4
source    0
target    0
weight    0
dtype: int64


In [5]:
print(f"Number of edges before dropping self-loops: {len(edge_list_df)}")

edge_list_df = edge_list_df[edge_list_df['source'] != edge_list_df['target']]

print(f"Number of edges after dropping self-loops: {len(edge_list_df)}")

Number of edges before dropping self-loops: 51377
Number of edges after dropping self-loops: 51373


In [6]:
# Find duplicate edges (ignoring the weight column)
duplicate_edges = edge_list_df.duplicated(subset=['source', 'target'], keep=False)

# Filter to get only the duplicate edges
parallel_edges_df = edge_list_df[duplicate_edges]

# Sort to better visualize parallel edges
parallel_edges_sorted = parallel_edges_df.sort_values(by=['source', 'target'])

print(parallel_edges_sorted)

Empty DataFrame
Columns: [source, target, weight]
Index: []


## Creating Undirected Weighted Graph

We iterate over the edge list DataFrame rows to add edges along with their weights to a new NetworkX graph.


In [7]:
import networkx as nx

def get_graph_info(graph):
    print("Number of nodes:", graph.number_of_nodes())
    print("Number of edges:", graph.number_of_edges())
    
    # Checking the graph type to provide appropriate information
    if isinstance(graph, nx.DiGraph):
        print("Graph is Directed")
    else:
        print("Graph is Undirected")


In [8]:
# Initialize a new graph
G = nx.MultiGraph()

# Add edges and weights
for index, row in edge_list_df.iterrows():
    source = row['source']
    target = row['target']
    weight = row['weight']
    
    # Add the edge with weight
    G.add_edge(source, target, weight=weight)

In [9]:
get_graph_info(G)

Number of nodes: 15649
Number of edges: 51373
Graph is Undirected


## Testing Modularity, Run Time and No. of Communities

- Iterates 10 times, running both the Louvain and Leiden using CDLib on each iteration.
- Records modularity, number of communities, and execution time for each run.
- Calculates the average modularity, average number of communities, and average execution time for both algorithms across all runs.
- Stores all this information in the results dictionary for easy access and analysis.

In [10]:
from cdlib import algorithms
import time
import numpy as np

# Prepare the data structure for results
results = {
    "Louvain": {"modularity": [], "communities": [], "time": []},
    "Leiden": {"modularity": [], "communities": [], "time": []}
}

# Execute each algorithm 10 times
for _ in range(10):
    # Louvain
    start_time = time.time()
    communities_louvain = algorithms.louvain(G, weight='weight')
    elapsed_time = time.time() - start_time
    results["Louvain"]["modularity"].append(communities_louvain.newman_girvan_modularity().score)
    results["Louvain"]["communities"].append(len(communities_louvain.communities))
    results["Louvain"]["time"].append(elapsed_time)
    
    # Leiden
    start_time = time.time()
    communities_leiden = algorithms.leiden(G, weights='weight')
    elapsed_time = time.time() - start_time
    results["Leiden"]["modularity"].append(communities_leiden.newman_girvan_modularity().score)
    results["Leiden"]["communities"].append(len(communities_leiden.communities))
    results["Leiden"]["time"].append(elapsed_time)

# Calculate averages
for method in results:
    results[method]["avg_modularity"] = np.mean(results[method]["modularity"])
    results[method]["avg_time"] = np.mean(results[method]["time"])
    results[method]["avg_communities"] = np.mean(results[method]["communities"])


Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'wurlitzer', 'graph_tool', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [11]:
# Display the results
print(results)

{'Louvain': {'modularity': [0.5800333293129618, 0.6045040024210548, 0.6029962247636718, 0.6066544990866413, 0.5675916203008841, 0.5804464258097656, 0.6009444677169751, 0.6113157722888504, 0.637735280274994, 0.5612300136034208], 'communities': [1841, 1709, 1726, 1751, 1970, 1832, 1745, 1654, 1480, 1907], 'time': [8.318804025650024, 7.781469821929932, 6.852544069290161, 6.3088109493255615, 6.539769887924194, 8.105946063995361, 5.631251811981201, 7.129033088684082, 6.118636131286621, 6.201063871383667], 'avg_modularity': 0.5953451635579219, 'avg_time': 6.898732972145081, 'avg_communities': 1761.5}, 'Leiden': {'modularity': [0.7193660643959252, 0.7170395808763892, 0.7228124598741187, 0.725754038114919, 0.7274157423487653, 0.7142585122586171, 0.7237329133587553, 0.7202067617237046, 0.7232845198078207, 0.7177469060694326], 'communities': [26, 27, 30, 29, 31, 28, 30, 29, 28, 31], 'time': [1.4310641288757324, 1.300278902053833, 1.4258060455322266, 1.2655999660491943, 1.4199981689453125, 1.4261

In [12]:
# Convert the results dictionary into a pandas DataFrame
# First, prepare the data in a structured form
data = {
    "Algorithm": [],
    "Run": [],
    "Modularity": [],
    "Communities": [],
    "Time (s)": []
}

# Populate the structured data from the results
for algo in results:
    for run in range(10):  # Assuming 10 runs as previously set
        data["Algorithm"].append(algo)
        data["Run"].append(run + 1)  # Run number (1-10)
        data["Modularity"].append(results[algo]["modularity"][run])
        data["Communities"].append(results[algo]["communities"][run])
        data["Time (s)"].append(results[algo]["time"][run])

# Creating the DataFrame
results_df = pd.DataFrame(data)

# Display the DataFrame for visual inspection
print(results_df)

# Additionally, creating a summary DataFrame for averages
summary_data = {
    "Algorithm": ["Louvain", "Leiden"],
    "Avg. Modularity": [results["Louvain"]["avg_modularity"], results["Leiden"]["avg_modularity"]],
    "Avg. Communities": [results["Louvain"]["avg_communities"], results["Leiden"]["avg_communities"]],
    "Avg. Time (s)": [results["Louvain"]["avg_time"], results["Leiden"]["avg_time"]]
}

summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)

   Algorithm  Run  Modularity  Communities  Time (s)
0    Louvain    1    0.580033         1841  8.318804
1    Louvain    2    0.604504         1709  7.781470
2    Louvain    3    0.602996         1726  6.852544
3    Louvain    4    0.606654         1751  6.308811
4    Louvain    5    0.567592         1970  6.539770
5    Louvain    6    0.580446         1832  8.105946
6    Louvain    7    0.600944         1745  5.631252
7    Louvain    8    0.611316         1654  7.129033
8    Louvain    9    0.637735         1480  6.118636
9    Louvain   10    0.561230         1907  6.201064
10    Leiden    1    0.719366           26  1.431064
11    Leiden    2    0.717040           27  1.300279
12    Leiden    3    0.722812           30  1.425806
13    Leiden    4    0.725754           29  1.265600
14    Leiden    5    0.727416           31  1.419998
15    Leiden    6    0.714259           28  1.426103
16    Leiden    7    0.723733           30  1.440816
17    Leiden    8    0.720207           29  1.

In [13]:
# Save the DataFrame to a CSV file
csv_filename = 'M2V_CD_Experiment_Values.csv'
results_df.to_csv(csv_filename, index=False)